In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict
from functools import reduce
import seaborn as sns
from scipy.stats import binned_statistic
from matplotlib.lines import Line2D

In [2]:
%cd ..

/nfs/homedirs/fuchsgru/MastersThesis


In [3]:
import data.constants as dc

In [4]:
collection_name = 'week19_ensemble_all_datasets'
collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    # print(ex['result'].keys())
    ex['metrics'] = ex['result']['results']

In [5]:
experiments_killed = [r for r in collection.find() if r['status'] in ('KILLED',)]

In [6]:
set(e['config']['data']['dataset'] for e in experiments_killed)

set()

In [7]:
experiments[0]['metrics'].keys()

dict_keys(['accuracy_val_val', 'accuracy_id_val_val', 'accuracy_ood_val_val', 'accuracy_val_no-edges_val', 'accuracy_id_val_no-edges_val', 'accuracy_ood_val_no-edges_val', 'ece_val_val', 'ece_val_no-edges_val', 'accuracy_ood-val_loc-no-edges_val', 'accuracy_id_ood-val_loc-no-edges_val', 'accuracy_ood_ood-val_loc-no-edges_val', 'ood_auroc_total-predictive-entropy_loc-no-edges_val', 'ood_aucpr_total-predictive-entropy_loc-no-edges_val', 'misclassification_auroc_total-predictive-entropy_loc-no-edges_val', 'misclassification_aucpr_total-predictive-entropy_loc-no-edges_val', 'ood_auroc_max-score_loc-no-edges_val', 'ood_aucpr_max-score_loc-no-edges_val', 'misclassification_auroc_max-score_loc-no-edges_val', 'misclassification_aucpr_max-score_loc-no-edges_val', 'ood_auroc_expected-softmax-entropy_loc-no-edges_val', 'ood_aucpr_expected-softmax-entropy_loc-no-edges_val', 'misclassification_auroc_expected-softmax-entropy_loc-no-edges_val', 'misclassification_aucpr_expected-softmax-entropy_loc-no

In [8]:
METRIC = 'Metric'
OOD_AUROC = 'OOD AUC-ROC'
OOD_AUCPR = 'OOD AUC-PR'
MISCLASSICIFACTION_AUROC = 'Misclassification AUC-ROC'
MISCLASSICIFACTION_AUCPR = 'Misclassification AUC-PR'

DATASET = 'Dataset'

SETTING = 'Setting'
HYBRID = 'Inductive'
TRANSDUCTIVE = 'Transductive'
LOC = 'Leave Out Classes'
NORMAL = 'Normal'
BERNOULLI = 'Bernoulli'
NO_EDGES = 'No Edges'
PROXY = 'Proxy'
EXPERIMENT = 'Experiment'

OOD_TYPE = 'OOD Type'


RESIDUAL = 'Residual'
SPECTRAL_NORM = 'Spectral Norm'
WEIGHT_SCALE = 'Weight Scale'
LOWER_LIPSCHITZ = 'Empirical Lower Lipschitz Bound'
UPPER_LIPSCHITZ = 'Empirical Upper Lipschitz Bound'



EPISTEMIC = 'Epistemic'
ALEATORIC = 'Aleatoric'

ACCURACY = 'Accuracy'
ACCURACY_ID = 'In-distribution Accuracy'
ECE = 'Expected Calibration Error'

MODEL = 'Model'
DROPOUT = 'Dropout'
DROP_EDGE = 'Drop Edge'

In [9]:
set(ex['config']['model']['residual'] for ex in experiments), \
    set(ex['config']['model']['use_spectral_norm'] for ex in experiments)

({False}, {False})

In [10]:
metrics = set()
for ex in experiments:
    for m in [m for m in ex['metrics'].keys() if 'accuracy' in m]:
        metrics.add(m)

list(metrics)

['accuracy_id_normal-val_normal_val',
 'accuracy_val_no-edges_val',
 'accuracy_id_ood-val_loc_val',
 'accuracy_test_no-edges_test',
 'accuracy_ber-test_ber-no-edges_test',
 'accuracy_id_ood-val_loc-no-edges_val',
 'accuracy_ood_ood-val_loc-no-edges_val',
 'accuracy_ood-val_loc-no-edges_val',
 'accuracy_ber-val_ber_val',
 'accuracy_id_val_val',
 'accuracy_id_test_test',
 'accuracy_ood_ber-val_ber_val',
 'accuracy_ood_normal-test_normal-no-edges_test',
 'accuracy_val_val',
 'accuracy_id_ber-test_ber-no-edges_test',
 'accuracy_id_ber-val_ber_val',
 'accuracy_ood_normal-val_normal_val',
 'accuracy_ber-val_ber-no-edges_val',
 'accuracy_ood_val_no-edges_val',
 'accuracy_ood_normal-test_normal_test',
 'accuracy_id_val_no-edges_val',
 'accuracy_normal-val_normal-no-edges_val',
 'accuracy_id_normal-test_normal_test',
 'accuracy_ood_normal-val_normal-no-edges_val',
 'accuracy_ood_ber-test_ber_test',
 'accuracy_id_normal-val_normal-no-edges_val',
 'accuracy_id_ber-test_ber_test',
 'accuracy_ood-v

In [11]:
mode = 'test'

In [13]:
data_proxy, data_acc_ece = [], []
for ex in experiments:
    cfg = ex['config']
    base = {
        SETTING : {dc.HYBRID : HYBRID, dc.TRANSDUCTIVE : TRANSDUCTIVE}[cfg['data']['setting']],
        DATASET : cfg['data']['dataset'],
    }
    data_acc_ece.append(base | {
        OOD_TYPE : cfg['data']['ood_type'],
        ACCURACY : ex['metrics'][f'accuracy_{mode}_{mode}'][0],
        ECE : ex['metrics'][f'ece_{mode}_{mode}'][0]['value'],
    })
    if cfg['data']['ood_type'] == dc.PERTURBATION:
        ood_types = (
            (BERNOULLI, 'ber'),
            (NORMAL, 'normal'),
        )
    elif cfg['data']['ood_type'] == dc.LEFT_OUT_CLASSES:
        ood_types = (
            (LOC, 'loc'),
        )
    else:
        raise ValueError(cfg['data']['ood_type'])
    
    for no_edge_suffix, no_edges in (('-no-edges', True), ('', False)):
    
        for ood_type, ood_name in ood_types:
            for proxy, proxy_name in ((ALEATORIC, 'expected-softmax-entropy'), (EPISTEMIC, 'mutual-information')):
                data_proxy += [
                    base | {
                        EXPERIMENT : ood_type,
                        PROXY : proxy,
                        OOD_AUROC : ex['metrics'][f'ood_auroc_{proxy_name}_{ood_name}{no_edge_suffix}_{mode}'][0]['value'],
                        OOD_AUCPR : ex['metrics'][f'ood_aucpr_{proxy_name}_{ood_name}{no_edge_suffix}_{mode}'][0]['value'],
                        MISCLASSICIFACTION_AUROC : ex['metrics'][f'misclassification_auroc_{proxy_name}_{ood_name}{no_edge_suffix}_{mode}'][0]['value'],
                        MISCLASSICIFACTION_AUCPR : ex['metrics'][f'misclassification_aucpr_{proxy_name}_{ood_name}{no_edge_suffix}_{mode}'][0]['value'],
                        NO_EDGES : no_edges,
                    }
                ]
    
        
        
    
df_proxy = pd.DataFrame(data_proxy)
df_acc_ece = pd.DataFrame(data_acc_ece)

In [14]:
df_acc_ece[
    (df_acc_ece[DATASET] == dc.CORA_FULL)
]

,Setting,Dataset,OOD Type,Accuracy,Expected Calibration Error
0,Transductive,cora_full,left-out-classes,0.851562,0.174319
1,Transductive,cora_full,left-out-classes,0.828063,0.168053
2,Transductive,cora_full,left-out-classes,0.825919,0.132658
3,Transductive,cora_full,left-out-classes,0.811765,0.153339
4,Transductive,cora_full,left-out-classes,0.856322,0.171735
...,...,...,...,...,...
95,Inductive,cora_full,perturbations,0.827014,0.161644
96,Inductive,cora_full,perturbations,0.808153,0.123655
97,Inductive,cora_full,perturbations,0.842956,0.171370
98,Inductive,cora_full,perturbations,0.828704,0.146517


In [16]:
def agg_mean_and_std(group):
    mean = group.mean()
    std = group.std()
    return f'{mean:.2f} ± {std:.2f}'
    return mean

df_proxy_agg = df_proxy.groupby([SETTING, DATASET, EXPERIMENT, NO_EDGES, PROXY]).agg(agg_mean_and_std)

In [20]:
df_tmp = df_proxy_agg.reset_index()
df_tmp = df_tmp.melt(id_vars=[SETTING, DATASET, EXPERIMENT, NO_EDGES, PROXY], value_vars = [OOD_AUROC, OOD_AUCPR, MISCLASSICIFACTION_AUROC, MISCLASSICIFACTION_AUCPR], var_name='Metric')
df_tmp = df_tmp[(df_tmp[METRIC] == OOD_AUROC) | (df_tmp[METRIC] == OOD_AUCPR)]
df_tmp = df_tmp.pivot(index=[SETTING, DATASET, PROXY], columns=[METRIC, EXPERIMENT, NO_EDGES]).T.sort_index().T
df_tmp.to_csv('~/ensemble_all_datasets_ood_detection.csv')

In [21]:
df_tmp

value               \
Metric                                OOD AUC-PR                
Experiment                             Bernoulli                
No Edges                                   False         True   
Setting      Dataset      Proxy                                 
Inductive    amazon_photo Aleatoric  0.50 ± 0.01  0.84 ± 0.08   
                          Epistemic  0.50 ± 0.03  0.75 ± 0.08   
             citeseer     Aleatoric  0.78 ± 0.03  0.99 ± 0.00   
                          Epistemic  0.69 ± 0.05  0.87 ± 0.07   
             coauthor_cs  Aleatoric  0.60 ± 0.02  1.00 ± 0.00   
                          Epistemic  0.58 ± 0.02  0.95 ± 0.03   
             cora_full    Aleatoric  0.61 ± 0.03  0.96 ± 0.02   
                          Epistemic  0.58 ± 0.06  0.96 ± 0.03   
             cora_ml      Aleatoric  0.61 ± 0.03  0.92 ± 0.04   
                          Epistemic  0.63 ± 0.07  1.00 ± 0.00   
             ogbn_arxiv   Aleatoric  0.54 ± 0.00  0.70 ± 0.00   
                          Epistemic  0.57 ± 0.02  0.94 ± 0.03   
             pubmed       Aleatoric  0.69 ± 0.02  0.83 ± 0.05   
                          Epistemic  0.69 ± 0.07  0.89 ± 0.08   
Transductive amazon_photo Aleatoric  0.50 ± 0.02  0.82 ± 0.07   
                          Epistemic  0.50 ± 0.02  0.74 ± 0.06   
             citeseer     Aleatoric  0.72 ± 0.06  1.00 ± 0.00   
                          Epistemic  0.49 ± 0.05  0.80 ± 0.07   
             coauthor_cs  Aleatoric  0.57 ± 0.02  1.00 ± 0.00   
                          Epistemic  0.55 ± 0.02  0.95 ± 0.02   
             cora_full    Aleatoric  0.57 ± 0.04  0.95 ± 0.01   
                          Epistemic  0.50 ± 0.05  0.96 ± 0.03   
             cora_ml      Aleatoric  0.57 ± 0.04  0.90 ± 0.05   
                          Epistemic  0.58 ± 0.09  0.99 ± 0.03   
             ogbn_arxiv   Aleatoric  0.53 ± 0.00  0.68 ± 0.00   
                          Epistemic  0.56 ± 0.01  0.92 ± 0.02   
             pubmed       Aleatoric  0.59 ± 0.03  0.78 ± 0.12   
                          Epistemic  0.59 ± 0.08  0.90 ± 0.11   

                                                                    \
Metric                                                               
Experiment                          Leave Out Classes                
No Edges                                        False         True   
Setting      Dataset      Proxy                                      
Inductive    amazon_photo Aleatoric       0.67 ± 0.04  0.65 ± 0.02   
                          Epistemic       0.85 ± 0.08  0.64 ± 0.04   
             citeseer     Aleatoric       0.66 ± 0.12  0.50 ± 0.07   
                          Epistemic       0.27 ± 0.10  0.31 ± 0.10   
             coauthor_cs  Aleatoric       0.85 ± 0.06  0.83 ± 0.05   
                          Epistemic       0.38 ± 0.09  0.67 ± 0.06   
             cora_full    Aleatoric       0.87 ± 0.03  0.76 ± 0.03   
                          Epistemic       0.21 ± 0.06  0.30 ± 0.06   
             cora_ml      Aleatoric       0.90 ± 0.04  0.67 ± 0.02   
                          Epistemic       0.27 ± 0.05  0.39 ± 0.07   
             ogbn_arxiv   Aleatoric       0.86 ± 0.00  0.38 ± 0.01   
                          Epistemic       0.82 ± 0.04  0.50 ± 0.02   
             pubmed       Aleatoric       0.24 ± 0.03  0.27 ± 0.03   
                          Epistemic       0.25 ± 0.07  0.26 ± 0.06   
Transductive amazon_photo Aleatoric       0.67 ± 0.11  0.66 ± 0.04   
                          Epistemic       0.86 ± 0.08  0.66 ± 0.05   
             citeseer     Aleatoric       0.61 ± 0.15  0.45 ± 0.14   
                          Epistemic       0.22 ± 0.13  0.27 ± 0.12   
             coauthor_cs  Aleatoric       0.66 ± 0.14  0.70 ± 0.08   
                          Epistemic       0.45 ± 0.10  0.61 ± 0.08   
             cora_full    Aleatoric       0.93 ± 0.02  0.80 ± 0.05   
                          Epistemic       0.13 ± 0.03  0.32 ± 0.06   
       

In [23]:
df_tmp = df_proxy_agg.reset_index()
df_tmp = df_tmp.melt(id_vars=[SETTING, DATASET, EXPERIMENT, NO_EDGES, PROXY], value_vars = [OOD_AUROC, OOD_AUCPR, MISCLASSICIFACTION_AUROC, MISCLASSICIFACTION_AUCPR], var_name='Metric')
df_tmp = df_tmp[(df_tmp[METRIC] == MISCLASSICIFACTION_AUROC) | (df_tmp[METRIC] == MISCLASSICIFACTION_AUCPR)]
df_tmp = df_tmp.pivot(index=[SETTING, DATASET, PROXY], columns=[METRIC, EXPERIMENT, NO_EDGES]).T.sort_index().T
df_tmp.to_csv('~/ensemble_all_datasets_misclassification_detection.csv')

In [24]:
df_tmp

value               \
Metric                              Misclassification AUC-PR                
Experiment                                         Bernoulli                
No Edges                                               False         True   
Setting      Dataset      Proxy                                             
Inductive    amazon_photo Aleatoric              0.97 ± 0.01  0.82 ± 0.05   
                          Epistemic              0.91 ± 0.02  0.76 ± 0.06   
             citeseer     Aleatoric              0.83 ± 0.03  0.78 ± 0.04   
                          Epistemic              0.66 ± 0.05  0.60 ± 0.05   
             coauthor_cs  Aleatoric              0.96 ± 0.00  0.94 ± 0.01   
                          Epistemic              0.93 ± 0.01  0.90 ± 0.02   
             cora_full    Aleatoric              0.92 ± 0.03  0.82 ± 0.03   
                          Epistemic              0.79 ± 0.07  0.80 ± 0.03   
             cora_ml      Aleatoric              0.91 ± 0.03  0.81 ± 0.03   
                          Epistemic              0.77 ± 0.06  0.79 ± 0.03   
             ogbn_arxiv   Aleatoric              0.77 ± 0.00  0.58 ± 0.00   
                          Epistemic              0.73 ± 0.01  0.63 ± 0.01   
             pubmed       Aleatoric              0.87 ± 0.01  0.79 ± 0.03   
                          Epistemic              0.77 ± 0.03  0.76 ± 0.05   
Transductive amazon_photo Aleatoric              0.97 ± 0.01  0.81 ± 0.04   
                          Epistemic              0.91 ± 0.02  0.77 ± 0.04   
             citeseer     Aleatoric              0.80 ± 0.04  0.70 ± 0.08   
                          Epistemic              0.67 ± 0.09  0.54 ± 0.07   
             coauthor_cs  Aleatoric              0.97 ± 0.01  0.94 ± 0.00   
                          Epistemic              0.95 ± 0.01  0.90 ± 0.02   
             cora_full    Aleatoric              0.91 ± 0.05  0.80 ± 0.07   
                          Epistemic              0.77 ± 0.06  0.78 ± 0.07   
             cora_ml      Aleatoric              0.90 ± 0.05  0.77 ± 0.06   
                          Epistemic              0.79 ± 0.04  0.78 ± 0.07   
             ogbn_arxiv   Aleatoric              0.78 ± 0.00  0.57 ± 0.00   
                          Epistemic              0.74 ± 0.02  0.62 ± 0.01   
             pubmed       Aleatoric              0.87 ± 0.02  0.74 ± 0.04   
                          Epistemic              0.80 ± 0.03  0.74 ± 0.04   

                                                                    \
Metric                                                               
Experiment                          Leave Out Classes                
No Edges                                        False         True   
Setting      Dataset      Proxy                                      
Inductive    amazon_photo Aleatoric       0.66 ± 0.04  0.63 ± 0.03   
                          Epistemic       0.76 ± 0.08  0.62 ± 0.04   
             citeseer     Aleatoric       0.43 ± 0.03  0.35 ± 0.04   
                          Epistemic       0.22 ± 0.05  0.24 ± 0.06   
             coauthor_cs  Aleatoric       0.70 ± 0.07  0.72 ± 0.05   
                          Epistemic       0.36 ± 0.07  0.61 ± 0.06   
             cora_full    Aleatoric       0.83 ± 0.07  0.75 ± 0.06   
                          Epistemic       0.18 ± 0.05  0.30 ± 0.07   
             cora_ml      Aleatoric       0.79 ± 0.06  0.59 ± 0.06   
                          Epistemic       0.21 ± 0.04  0.33 ± 0.09   
             ogbn_arxiv   Aleatoric       0.62 ± 0.00  0.34 ± 0.01   
                          Epistemic       0.56 ± 0.02  0.38 ± 0.01   
             pubmed       Aleatoric       0.24 ± 0.02  0.29 ± 0.02   
                          Epistemic       0.24 ± 0.03  0.26 ± 0.03   
Transductive amazon_photo Aleatoric       0.64 ± 0.08  0.63 ± 0.04   
                          Epistemic       0.77 ± 0.08  0.63 ± 0.04   
             citeseer     Aleatoric       0